In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
IMG_DIR = 'camera_cal'
NX, NY = 9, 6  # number of points on a chessboard

objpoints = []  # 3d points in real world space
imgpoints = []  # 2d points in image plane.

objp = np.zeros((NX * NY, 3), np.float32)
objp[:, :2] = np.mgrid[0:NX, 0:NY].T.reshape(-1, 2)

# get the list of calibration images
images = [os.path.join(IMG_DIR, filename) for filename in os.listdir(IMG_DIR)]

for image in images:
    img = cv2.imread(image)

    # convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (NX, NY), None)
    
    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)

In [ ]:
test_image = 'camera_cal/calibration1.jpg'
img = cv2.imread(test_image)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, (img.shape[1], img.shape[0]), None, None)
dst = cv2.undistort(img, mtx, dist, None, mtx)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=20)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=20)

In [ ]:
test_image = 'test_images/test1.jpg'
img = cv2.imread(test_image)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, (img.shape[1], img.shape[0]), None, None)
dst = cv2.undistort(img, mtx, dist, None, mtx)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=20)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=20)

In [ ]:
"""
test_image = 'test_images/straight_line1.jpg'
img = cv2.imread(test_image)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
from matplotlib.path import Path
import matplotlib.patches as patches
height = img.shape[0]

src = np.float32([(190, 719), (577, 460), (705, 460), (1120, 719)])
plt.figure(figsize=(20, 10))
plt.imshow(img)
plt.plot([src[i][0] for i in range(4)],
         [src[i][1] for i in range(4)],
         'r-', lw=2)
 """

In [ ]:
test_image = 'test_images/straight_lines1.jpg'
img = cv2.imread(test_image)
#src = np.float32([(190, 719), (577, 460), (705, 460), (1120, 719)])
#dst = np.float32([(190, 719), (190, 0), (1120, 0), (1120, 719)])
src = np.float32([(190, 719), (577, 460), (705, 460), (1120, 719)])
dst = np.float32([(190, 719), (190, 0), (1120, 0), (1120, 719)])
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)
warped = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))
plt.figure(figsize=(20, 10))
plt.imshow(cv2.cvtColor(warped, cv2.COLOR_BGR2RGB))
plt.plot([dst[i][0] for i in range(4)], [dst[i][1] for i in range(4)], 'r-', lw=2)

In [ ]:
right_fit = None
left_fit = None
AV = 20
lcache = []
rcache = []
no_lanes = 0

def curvature(ploty, fit):
    y = np.max(ploty)
    curverad = ((1 + (2 * fit[0] * y + fit[1]) ** 2) ** 1.5) / np.absolute(2 * fit[0])
    return int(curverad)
    

def pipeline(image, mtx, dist, M, Minv, cache=True, debug=False):
    global lcache, rcache
    global frame
    global no_lanes
    img = np.copy(image)
    img = cv2.undistort(img, mtx, dist, None, mtx)

    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    s = hls[:, :, 2]
    #thresh = (150, 255)
    thresh = (150, 255)
    s_binary = np.zeros_like(s)
    s_binary[(s >= thresh[0]) & (s <= thresh[1])] = 1

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sobel = cv2.Sobel(s, cv2.CV_64F, 1, 0)
    sobel = np.absolute(sobel)
    sobel = np.uint8(255 * sobel / np.max(sobel))
    thresh = (30, 100)
    sobel_binary = np.zeros_like(sobel)
    sobel_binary[(sobel >= thresh[0]) & (sobel <= thresh[1])] = 1

    combined_binary = np.zeros_like(s_binary)
    combined_binary[(s_binary == 1) | (sobel_binary == 1)] = 1
    #combined_binary[(s_binary == 1) ] = 1
    #combined_binary[(s_binary == 1)] = 1

    #plt.imshow(combined_binary, cmap='gray')
    warped = cv2.warpPerspective(combined_binary, M, (img.shape[1], img.shape[0]))
    #plt.imshow(warped, cmap='gray')
    
    histogram = np.sum(warped[warped.shape[0] // 2:,:], axis=0)
    #print(histogram)
    #plt.plot(histogram)
    S = np.sum(warped[warped.shape[0] // 2:,:])
    #print(S)
    if S > 50000:
        thresh = (120, 255)
        g = img[:, :, 0]
        g_binary = np.zeros_like(g)
        g_binary[(g >= thresh[0]) & (g <= thresh[1])] = 1

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        sobel = cv2.Sobel(s, cv2.CV_64F, 1, 0)
        sobel = np.absolute(sobel)
        sobel = np.uint8(255 * sobel / np.max(sobel))
        thresh = (30, 100)
        sobel_binary = np.zeros_like(sobel)
        sobel_binary[(sobel >= thresh[0]) & (sobel <= thresh[1])] = 1

        combined_binary = np.zeros_like(g_binary)
        combined_binary[(g_binary == 1) | (sobel_binary == 1)] = 1
        #combined_binary[(g_binary == 1) ] = 1
        #combined_binary[(sobel_binary == 1)] = 1

        #plt.imshow(combined_binary, cmap='gray')
        warped = cv2.warpPerspective(combined_binary, M, (img.shape[1], img.shape[0]))
        histogram = np.sum(warped[warped.shape[0] // 2:,:], axis=0)
        S = np.sum(warped[warped.shape[0] // 2:,:])
        #print(S)
    #plt.imshow(warped, cmap='gray')
        
    out_img = np.dstack((warped, warped, warped)) * 255
    midpoint = np.int(histogram.shape[0] / 2)
    #print(midpoint)
    lx_base = np.argmax(histogram[:midpoint])
    rx_base = np.argmax(histogram[midpoint:]) + midpoint
    #print(rx_base)
    #plt.plot(histogram)
    #return

    n_windows = 9
    window_height = np.int(img.shape[0] / n_windows)

    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = warped[:680, :].nonzero()
    #print(warped)
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    # Current positions to be updated for each window
    lx_current = lx_base
    rx_current = rx_base

    # Set the width of the windows +/- margin
    margin = 100

    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Create empty lists to receive left and right lane pixel indices
    l_indices = []
    r_indices = []
    #if frame > 5:
    #    if len(rcache) <= 4:
    #print(frame, rcache)
    #        #assert False
    frame += 1
        

    if not lcache or not rcache or no_lanes > 1:
        # Step through the windows one by one
        for w in range(n_windows):
            # Identify window boundaries in x and y (and right and left)
            y = (img.shape[0] - (w + 1) * window_height,
                 img.shape[0] - w * window_height)
            x = ((lx_current - margin, lx_current + margin),  # left low, left high
                 (rx_current - margin, rx_current + margin))  # right low, right high

            # Identify the nonzero pixels in x and y within the window
            good = []
            for side in (0, 1):
                # Draw the windows on the visualization image
                cv2.rectangle(out_img, (x[side][0], y[0]), (x[side][1], y[1]), (0, 255, 0), 2)
                good.append(((nonzeroy >= y[0]) & (nonzeroy < y[1]) & (nonzerox >= x[side][0]) & (nonzerox < x[side][1])).nonzero()[0])
            # Append these indices to the lists
            l_indices.append(good[0])
            r_indices.append(good[1])
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good[0]) > minpix:
                lx_current = np.int(np.mean(nonzerox[good[0]]))
            if len(good[1]) > minpix:        
                rx_current = np.int(np.mean(nonzerox[good[1]]))

        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(l_indices)
        right_lane_inds = np.concatenate(r_indices)
    else:
        left_fit = lcache[-1]
        right_fit = rcache[-1]
        left_lane_inds = ((nonzerox > (left_fit[0] * (nonzeroy ** 2) + left_fit[1] * nonzeroy + left_fit[2] - 3 * margin))
                          & (nonzerox < (left_fit[0] * (nonzeroy ** 2) + left_fit[1] * nonzeroy + left_fit[2] + 3 * margin))) 
        right_lane_inds = ((nonzerox > (right_fit[0] * (nonzeroy ** 2) + right_fit[1] * nonzeroy + right_fit[2] - 3 * margin))
                           & (nonzerox < (right_fit[0] * (nonzeroy ** 2) + right_fit[1] * nonzeroy + right_fit[2] + 3 * margin)))  
        if not left_lane_inds.size or right_lane_inds.size:
            no_lanes += 1

        

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    """
    DIFF = 0.01
    if len(leftx) >= 50 and leftx.max() - leftx.min() > window_height:
        left_fit = np.polyfit(lefty, leftx, 2)
        if not lcache or np.linalg.norm(left_fit - np.mean(lcache, 0)) < DIFF:
            lcache.append(left_fit)
    if len(rightx) >= 10 and rightx.max() - rightx.min() > window_height:
        right_fit = np.polyfit(righty, rightx, 2)
        if not rcache or np.linalg.norm(right_fit - np.mean(rcache, 0)) < DIFF:
            rcache.append(right_fit)
    if len(lcache) > AV:
        lcache = lcache[len(lcache) - AV:]
    if len(rcache) > AV:
        rcache = lcache[len(rcache) - AV:]
    """
    DIFF = 1000
    if lefty.size and leftx.size:
        left_fit = np.polyfit(lefty, leftx, 2)
        close = True
        if lcache:
            ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0])
            curr = curvature(ploty, left_fit)
            prev = curvature(ploty, lcache[-1])
            #close = abs(curr - prev) < DIFF

        if len(leftx) >= 50 and lefty.max() - lefty.min() > window_height and close:
            lcache.append(left_fit)
    
    if righty.size and rightx.size:
        right_fit = np.polyfit(righty, rightx, 2)
        close = True
        if rcache:
            ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0])
            curr = curvature(ploty, right_fit)
            prev = curvature(ploty, rcache[-1])
            #close = abs(curr - prev) < DIFF

        if len(rightx) >= 50 and righty.max() - righty.min() > window_height and close:
            rcache.append(right_fit)
    if len(lcache) > AV:
        lcache = lcache[len(lcache) - AV:]
    if len(rcache) > AV:
        #print(rcache)
        rcache = rcache[len(rcache) - AV:]
        #print(rcache)

    #if cache:
    #print('calculated {}'.format(right_fit))
    left_fit = np.mean(lcache, 0)
    right_fit = np.mean(rcache, 0)


    #print('average {}'.format(np.mean(rcache, 0)))


   
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0])
    left_fitx = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]
    
    if debug:
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
        plt.imshow(out_img)
        plt.plot(left_fitx, ploty, color='yellow')
        plt.plot(right_fitx, ploty, color='yellow')
        plt.xlim(0, 1280)
        plt.ylim(720, 0)

    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    #for p in pts[0]:
    #    print(p)

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))
    #plt.imshow(color_warp)

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0]))
    #plt.imshow(newwarp)
    # Combine the result with the original image

    result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)
    cv2.putText(result, "left rad: {}".format(curvature(ploty, left_fit)),
                (50, 100), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 0, 255))
    cv2.putText(result, "right rad: {}".format(curvature(ploty, right_fit)),
                (650, 100), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 0, 255))

    #plt.imshow(result)
    return result

In [ ]:
AV = 1
import glob
images = glob.glob('test_images/test*')
plt.subplots(figsize=(10, 30))
no_lanes = 0
frame = 0
lcache = []
rcache = []
for i, test_image in enumerate(images):
    img = cv2.imread(test_image)
    plt.subplot(len(images), 1, i + 1)
    plt.imshow(cv2.cvtColor(pipeline(img, mtx, dist, M, Minv), cv2.COLOR_BGR2RGB))


In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image, debug=False):
    #try:
    result = pipeline(image, mtx, dist, M, Minv, debug)
    return result
    #except:
    #    return np.copy(image)

In [ ]:
#output = 'project_video_result.mp4'
#clip1 = VideoFileClip("project_video.mp4")
#clip = clip1.fl_image(process_image)
#%time clip.write_videofile(output, audio=False)

In [ ]:
#HTML("""
#<video width="960" height="540" controls>
#  <source src="{0}">
#</video>
#""".format(output))

In [ ]:
#plt.subplots(figsize=(15, 8))
#img = clip.get_frame(24.5)
#plt.imshow(img)

In [ ]:
#plt.subplots(figsize=(15, 8))
#img = clip1.get_frame(24.5)
#plt.imshow(img)
#processed = process_image(img)

In [ ]:
right_fit = None
left_fit = None
lcache = []
rcache = []
no_lanes = 0
frame = 0
AV = 5
output = 'project_video_result.mp4'
clip1 = VideoFileClip("project_video.mp4")#.subclip(40, 45)
clip = clip1.fl_image(process_image)
%time clip.write_videofile(output, audio=False)

In [ ]:
clip2 = VideoFileClip(output)
clip2.ipython_display()

In [ ]:
plt.subplots(figsize=(15, 8))
img = clip1.get_frame(1.5)
#plt.imshow(img)
lcache = []
rcache = []
AV = 1
processed = pipeline(img, mtx, dist, M, Minv, debug=True)
#plt.imshow(processed)

In [ ]:
plt.subplots(figsize=(15, 8))
img = clip2.get_frame(1.6)
plt.imshow(img)

In [ ]:
img = clip1.get_frame(1.5)
hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(30, 10))

for i in range(3):
    ax = plt.subplot(2, 3, i + 1)
    ax.set_title('RGB'[i], fontsize=20)
    ax.imshow(rgb[:, :, i], cmap='gray')

for i in range(3):
    ax = plt.subplot(2, 3, i + 4)
    ax.set_title('HLS'[i], fontsize=20)
    ax.imshow(hls[:, :, i], cmap='gray')